In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/home/dyd/anaconda3/envs/vault/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the module from /home/dyd/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Sun Sep  3 21:15:00 2023) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.
Map: 100%|██████████| 408/408 [00:00<00:00, 3943.44 examples/s]


In [2]:
from transformers import TrainingArguments


output_dir = "/media/dyd/UDISK/output_model/"
training_args = TrainingArguments(output_dir, num_train_epochs=6)


In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# The head of the pretrained model has been discard and a new head suitable
# for sequence classification has been added instead.

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [4]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

/home/dyd/anaconda3/envs/vault/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 18%|█▊        | 500/2754 [00:40<02:33, 14.65it/s]

{'loss': 0.4984, 'learning_rate': 4.0922294843863475e-05, 'epoch': 1.09}


 36%|███▋      | 1000/2754 [02:12<01:58, 14.75it/s] 

{'loss': 0.2983, 'learning_rate': 3.184458968772695e-05, 'epoch': 2.18}


 54%|█████▍    | 1500/2754 [03:41<01:24, 14.86it/s]  

...

 91%|█████████ | 2500/2754 [06:47<00:17, 14.81it/s]  

{'loss': 0.0219, 'learning_rate': 4.611474219317357e-06, 'epoch': 5.45}


100%|██████████| 2754/2754 [08:02<00:00,  5.71it/s]

{'train_runtime': 482.1983, 'train_samples_per_second': 45.641, 'train_steps_per_second': 5.711, 'train_loss': 0.18928970343493515, 'epoch': 6.0}


TrainOutput(global_step=2754, training_loss=0.18928970343493515, metrics={'train_runtime': 482.1983, 'train_samples_per_second': 45.641, 'train_steps_per_second': 5.711, 'train_loss': 0.18928970343493515, 'epoch': 6.0})